In [ ]:
import math

# -------------------------
# Q1: Treasury bond price
# -------------------------
def bond_price(face=100.0, coupon_rate=0.10, ytm=0.02, years=25, freq=2):
    """
    Price a plain-vanilla coupon bond.
    Returns price per 'face' of par value.
    """
    n = int(years * freq)
    r = ytm / freq
    c = face * coupon_rate / freq  # coupon per period

    pv_coupons = c * (1 - (1 + r) ** (-n)) / r
    pv_face = face * (1 + r) ** (-n)
    return pv_coupons + pv_face

p1 = bond_price(face=100, coupon_rate=0.10, ytm=0.02, years=25, freq=2)
print("Q1 bond price per $100 face:", round(p1, 2))
print("Q1 bond price per $1,000 face:", round(p1 * 10, 2))

In [ ]:
# Q2: Future value
# -------------------------
def future_value_lump_sum(pv, annual_rate, years, freq=1):
    """
    Future value of a single deposit with compounding freq times/year.
    """
    n = int(years * freq)
    r = annual_rate / freq
    return pv * (1 + r) ** n

pv = 25000
fv_annual = future_value_lump_sum(pv, 0.04, 10, freq=1)
fv_monthly = future_value_lump_sum(pv, 0.04, 10, freq=12)

print("\nQ2 FV @4% compounded annually:", round(fv_annual, 2))
print("Q2 FV @4% compounded monthly:", round(fv_monthly, 2))


In [ ]:

# -------------------------
# Q3: Down payment + monthly annuity FV
# -------------------------
def future_value_annuity(payment, annual_rate, years, freq=12, annuity_due=False):
    """
    Future value of an annuity.
    - annuity_due=False: payments at END of period (ordinary annuity)
    - annuity_due=True : payments at BEGINNING of period (annuity due)
    """
    n = int(years * freq)
    r = annual_rate / freq
    fv = payment * ((1 + r) ** n - 1) / r
    if annuity_due:
        fv *= (1 + r)
    return fv

down_payment = 180000
pmt = 7000
annual_rate = 0.055
years = 20

fv_down = future_value_lump_sum(down_payment, annual_rate, years, freq=12)
fv_pmts = future_value_annuity(pmt, annual_rate, years, freq=12, annuity_due=False)  # end of month
fv_total = fv_down + fv_pmts

print("\nQ3 FV of down payment:", round(fv_down, 2))
print("Q3 FV of monthly payments:", round(fv_pmts, 2))
print("Q3 Total FV after 20 years:", round(fv_total, 2))

In [ ]:
# Helpers
# -----------------------------
def payment_from_pv(pv, annual_rate, n_months):
    """Monthly payment for an amortizing loan given PV, nominal annual rate, and term in months."""
    r = annual_rate / 12.0
    if abs(r) < 1e-12:
        return pv / n_months
    return pv * r / (1 - (1 + r) ** (-n_months))

def pv_of_annuity(pmt, monthly_rate, n_months):
    """Present value of level monthly payments."""
    r = monthly_rate
    if abs(r) < 1e-12:
        return pmt * n_months
    return pmt * (1 - (1 + r) ** (-n_months)) / r

def solve_monthly_irr_for_annuity(pmt, n_months, pv_target, lo=0.0, hi=1.0, tol=1e-12, max_iter=200):
    """
    Solve for monthly rate r such that PV(payments at r) = pv_target.
    Uses bisection (robust).
    """
    def f(r):
        return pv_of_annuity(pmt, r, n_months) - pv_target

    # Expand hi until sign changes (if needed)
    flo = f(lo)
    fhi = f(hi)
    while flo * fhi > 0:
        hi *= 2
        fhi = f(hi)
        if hi > 100:  # absurdly high monthly rate guard
            raise RuntimeError("Could not bracket IRR. Check inputs.")

    for _ in range(max_iter):
        mid = (lo + hi) / 2
        fmid = f(mid)
        if abs(fmid) < tol:
            return mid
        if flo * fmid <= 0:
            hi = mid
            fhi = fmid
        else:
            lo = mid
            flo = fmid
    return (lo + hi) / 2

def rate_conversions(monthly_rate):
    """Return nominal APR, effective APR (EAR) given monthly rate."""
    apr_nominal = monthly_rate * 12
    ear = (1 + monthly_rate) ** 12 - 1
    return apr_nominal, ear

In [ ]:
#Q4

cash_price = 513_000
dealer_price = 575_000
down_payment = 125_000
n = 36
dealer_stated_rate = 0.04  # "only a 4 percent interest rate"

# Dealer contract payment is based on financed amount at stated 4%
pv_dealer_contract = dealer_price - down_payment
pmt_dealer = payment_from_pv(pv_dealer_contract, dealer_stated_rate, n)

# "Effective cost" relative to true cash price (amount you'd otherwise need to finance)
pv_true_financed = cash_price - down_payment

# Solve for monthly IRR that makes PV(36 payments) = true financed amount
r_m_q4 = solve_monthly_irr_for_annuity(pmt_dealer, n, pv_true_financed)
apr_q4, ear_q4 = rate_conversions(r_m_q4)

print("Q4:")
print(f"  Dealer contract financed PV: ${pv_dealer_contract:,.2f}")
print(f"  Dealer monthly payment (stated 4%): ${pmt_dealer:,.2f}")
print(f"  True amount financed (cash price - down): ${pv_true_financed:,.2f}")
print(f"  Effective monthly IRR: {r_m_q4:.8f}")
print(f"  Effective nominal APR: {apr_q4:.6%}")
print(f"  Effective EAR:         {ear_q4:.6%}"
print()

In [ ]:
# =========================================================
# Q5
# =========================================================
# Existing loan:
old_balance = 800_000
old_pmt = 7_197.81
old_n = 240  # 20 years monthly

# New combined loan:
new_principal = 1_400_000
new_pmt = 15_415.21
new_n = 240

# Extra cash borrowed today (net proceeds):
extra_borrowed = 600_000

# Incremental payment stream attributable to borrowing extra + refinancing rate change:
delta_pmt = new_pmt - old_pmt

# Marginal monthly rate solves PV(delta payments) = extra borrowed
r_m_q5 = solve_monthly_irr_for_annuity(delta_pmt, new_n, extra_borrowed)
apr_q5, ear_q5 = rate_conversions(r_m_q5)

print("Q5:")
print(f"  Extra borrowed at t=0: ${extra_borrowed:,.2f}")
print(f"  Incremental monthly payment: ${delta_pmt:,.2f}")
print(f"  Effective monthly IRR (marginal): {r_m_q5:.8f}")
print(f"  Effective nominal APR:            {apr_q5:.6%}")
print(f"  Effective EAR:                    {ear_q5:.6%}")

In [ ]:
# Helper: PV of a growing annuity (payments at end of each period)
# PV = P0/(r-g) * (1 - ((1+g)/(1+r))^n)
# Solve for P0 given PV
# -----------------------------
def initial_payment_from_growing_annuity_pv(PV, r, g, n):
    if abs(r - g) < 1e-12:
        # Limit case r ~ g: PV = P0 * n / (1+r)
        return PV * (1 + r) / n
    return PV * (r - g) / (1 - ((1 + g) / (1 + r)) ** n)

# -----------------------------
# Helper: solve remaining months in an amortizing loan
# PV = PMT * (1 - (1+r)^(-n)) / r  => n = -ln(1 - r*PV/PMT)/ln(1+r)
# -----------------------------
def remaining_months_from_balance(PV, pmt, monthly_rate):
    r = monthly_rate
    if r <= 0:
        return PV / pmt
    x = 1 - r * PV / pmt
    if x <= 0:
        raise ValueError("Payment too small to amortize the balance (would never pay off).")
    return -math.log(x) / math.log(1 + r)

In [ ]:
# Q6
# =============================
loan = 1_000_000
contract_rate_annual = 0.06
payment_growth_annual = 0.02
n_months = 30 * 12

r_m = contract_rate_annual / 12
g_m = payment_growth_annual / 12

P0 = initial_payment_from_growing_annuity_pv(loan, r_m, g_m, n_months)

print("Q6:")
print(f"  Initial monthly payment: ${P0:,.2f}")
print()

# =============================
# Q7
# =============================
balance = 104_138.70
pmt = 5_000.00
contract_rate_annual = 0.14

r_m = contract_rate_annual / 12
n_remaining = remaining_months_from_balance(balance, pmt, r_m)

print("Q7:")
print(f"  Remaining months: {n_remaining:.6f}")
print(f"  Approx whole months: {round(n_remaining)}")
print(f"  Approx years: {n_remaining/12:.4f}")
print()

In [ ]:
# Q8 (Preferred stock as perpetuity)
# =============================
coupon_semiannual = 6.25
discount_rate_annual = 0.07
r_sa = discount_rate_annual / 2

price_pref = coupon_semiannual / r_sa

print("Q8:")
print(f"  Preferred stock price: ${price_pref:,.2f}")
print()

# =============================
# Q9 (Gordon Growth Model)
# =============================
D1 = 10.0          # dividend at end of next year
g = 0.03           # perpetual growth
r = 0.12           # required return

price_common = D1 / (r - g)

print("Q9:")
print(f"  Stock price (Gordon growth): ${price_common:,.2f}")
print()

In [ ]:
# Q10 (Grow 10% for 5 years, then constant to year 30)
# =============================
C1 = 1000.0
g = 0.10
r = 0.035

# PV of years 1..5 growing
pv_1_5 = sum(C1 * (1 + g) ** (t - 1) / (1 + r) ** t for t in range(1, 6))

# Level payment from years 6..30 equals C5 (payment at year 5)
C5 = C1 * (1 + g) ** 4
pv_6_30 = sum(C5 / (1 + r) ** t for t in range(6, 31))

pv_total = pv_1_5 + pv_6_30

print("Q10:")
print(f"  PV years 1-5:  ${pv_1_5:,.2f}")
print(f"  PV years 6-30: ${pv_6_30:,.2f}")
print(f"  Total PV:      ${pv_total:,.2f}")